In [ ]:
!nvidia-smi

In [ ]:
import os
import pickle
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [ ]:
os.getcwd()

In [ ]:
import pickle
import torch
with open('/common/home/apc120/Downloads/labelled/labelled_data.pkl', 'rb') as f:
    obj = f.read()
    data = pickle.loads(obj, encoding='latin1')


with open('/common/home/apc120/Downloads/labelled/embeddings_val1.pkl', 'rb') as f:
    val_emb = pickle.load(f)
val_img = val_emb[0]
val_txt = val_emb[1]

with open('/common/home/apc120/Downloads/labelled/embeddings_test1.pkl', 'rb') as f:
    test_emb = pickle.load(f)
test_img = torch.tensor(test_emb[0]).to('cuda')
test_txt = torch.tensor(test_emb[1]).to('cuda')

In [ ]:
def pickle_load(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [ ]:
val_txt = val_emb[1]
val_img = val_emb[0]

In [ ]:
import torch
from torch.utils.data import DataLoader
class EmbeddingDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    
    def __len__(self):
        return self.dataset[0].shape[0]
    
    def __getitem__(self, idx):
        text, img = self.dataset[1][idx], self.dataset[0][idx]
        return text, img

dataset = EmbeddingDataset(data)
data_loader = DataLoader(dataset, batch_size=64, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class TextEncoder(nn.Module):
    def __init__(self,output_size,input_size=1024):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),
            nn.LeakyReLU()
        )
        self.output = nn.Linear(512, output_size)
    
    def forward(self, x):
        x = self.layers(x)
        return self.output(x)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class ImgEncoder(nn.Module):
    def __init__(self,output_size,input_size=1024):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.BatchNorm1d(512),
            nn.Dropout(),
            nn.LeakyReLU()
        )
        self.output = nn.Linear(512, output_size)
    
    def forward(self, x):
        x = self.layers(x)
        return self.output(x)

In [ ]:
latent_dims = 32
img_model = ImgEncoder(latent_dims).to('cuda') # torch.load('step2/img_model.pth')
txt_model = TextEncoder(latent_dims).to('cuda') # torch.load('step2/text_model.pth')
optimizer_txt = torch.optim.Adam(txt_model.parameters(), lr=1e-6, weight_decay=1e-7)
optimizer_img = torch.optim.Adam(img_model.parameters(), lr=1e-6, weight_decay=1e-7)

# scheduler_txt = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_txt, mode='max', patience=3)
# scheduler_img = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_img, mode='max', patience=3)

# scheduler_txt = torch.optim.lr_scheduler.MultiStepLR(optimizer_txt, milestones=[30, 35], gamma=0.1, verbose=True)
# scheduler_img = torch.optim.lr_scheduler.MultiStepLR(optimizer_img, milestones=[30, 35], gamma=0.1, verbose=True)
criterion = nn.MSELoss() # lambda x, y : torch.sum(1 - nn.CosineSimilarity()(x, y))

In [ ]:
import random
import numpy as np
import torch
def calculate_rank_recall(txt_data, img_data):
    results_dict = {}
    projection_txt, projection_img = txt_data, img_data
    
    idxs = range(1000)
    
    glob_rank = []
    glob_recall = {1:0.0,5:0.0,10:0.0}
    
    for i in range(10):
        ids = random.sample(range(0,txt_data.shape[0]-1), 1000)
        
        txt_sample = projection_txt[ids,:]
        img_sample = projection_img[ids,:]
        
        similarity = np.dot(txt_sample.cpu().numpy(), img_sample.T.cpu().numpy())

        med_rank = []
        
        recall = {1:0.0,5:0.0,10:0.0}
        
        for ii in idxs:
            # get a column of similarities
            sim = similarity[ii,:]
            # sort indices in descending order
            sorting = np.argsort(sim)[::-1].tolist()
            # find where the index of the pair sample ended up in the sorting
            pos = sorting.index(ii)  
            if (pos+1) == 1:
                recall[1]+=1
            if (pos+1) <=5:
                recall[5]+=1
            if (pos+1)<=10:
                recall[10]+=1
            # store the position
            med_rank.append(pos+1)
        for i in recall.keys():
            recall[i]=recall[i]/1000
        med = np.median(med_rank)
        for i in recall.keys():
            glob_recall[i]+=recall[i]
        glob_rank.append(med)

    for i in glob_recall.keys():
        glob_recall[i] = glob_recall[i]/10
    med_dict = {}
    med_dict["mean_median"] = np.average(glob_rank)
    med_dict["recall"] = glob_recall
    med_dict["median_all"] = glob_rank
    print("Result:",med_dict)
    
    return med_dict

In [ ]:
import time
from tqdm import tqdm
def train(train_loader, img_model, txt_model, criterion, optimizer_txt, optimizer_img, epoch):
    print('Starting training epoch {}'.format(epoch))
    img_model.train()
    txt_model.train()   
    running_loss = []
    optimizer_txt.zero_grad()
    optimizer_img.zero_grad()
    
    for i, (txt_emb,img_emb) in tqdm(enumerate(train_loader)):
        
        txt_emb = txt_emb.to('cuda')
        img_emb = img_emb.to('cuda')
        
        optimizer_txt.zero_grad()
        optimizer_img.zero_grad()
        
        out_txt_emb = txt_model(txt_emb)
        out_img_emb = img_model(img_emb) 
        
        
        ######
        
        loss = criterion(out_txt_emb, out_img_emb)
        
        
        loss.backward()
        optimizer_txt.step()
        optimizer_img.step()
        
        running_loss.append(loss.item())
    return sum(running_loss)/len(running_loss)

In [ ]:
EPOCHS = 10
best_loss = 1e18
res_dict={}
for epoch in range(EPOCHS):
    img_model.train()
    txt_model.train()
    train_loss = train(data_loader, img_model, txt_model, criterion, optimizer_txt, optimizer_img, epoch)
    print('  Epoch {} loss: {}'.format(epoch, train_loss))
    med_dict = None    
    with torch.no_grad():
        txt_model.eval()
        img_model.eval()
        out_text, out_img = txt_model(torch.tensor(val_txt).to('cuda')), img_model(torch.tensor(val_img).to('cuda'))
        med_dict = calculate_rank_recall(out_text,out_img)
    if med_dict['mean_median'] < best_loss:
        print("BEST EPOCH:",epoch+1)
        torch.save(img_model, '32_mse_best_img_model.pth')
        torch.save(txt_model, '32_mse_best_txt_model.pth')
        best_loss = med_dict['mean_median']
#     scheduler_img.step(train_loss)
#     scheduler_txt.step(train_loss)
    med_dict["loss"] = train_loss
    res_dict["Epoch"+str(epoch+1)] = med_dict
    print('------------')

In [ ]:
dim_arr = [32]
for dim in dim_arr:
    img_model=torch.load(f'{dim}_mse_best_img_model.pth')
    txt_model = torch.load(f'{dim}_mse_best_txt_model.pth')
    with torch.no_grad():
        img_model.eval()
        txt_model.eval()    
        test_result = calculate_rank_recall(torch.tensor(txt_model(test_txt).to('cuda')),torch.tensor(img_model(test_img).to('cuda')))
        res_dict["test_result"] = test_result

In [ ]:
import json
with open("MSEresults_all_data.json", "w") as outfile:
    json.dump(res_dict, outfile)